For ranging markets: Bollinger Bands + Stochastic Oscillator
This combination is ideal for identifying potential reversals and entry points in markets that are not trending strongly. 
Bollinger Bands: Measure volatility and identify potential overbought or oversold price extremes within a range.
Usage: The price touching the outer bands can signal that the market is overextended.
Stochastic Oscillator: A momentum indicator that signals overbought and oversold conditions.
Usage: When the price is at the lower Bollinger Band and the Stochastic is in the oversold region (below 20), it could signal a buying opportunity. 

In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279221,85.324368,83.909697,84.556835,118944600
1,2010-01-05,85.504944,85.542570,84.918006,85.226528,111579900
2,2010-01-06,85.565170,85.775865,85.354475,85.422196,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212303,86.249930,85.527544,85.700613,126402800


In [3]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [4]:
# Calculate Bollinger Bands
df['BB_upper_band'] = [r.upper_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_middle_band'] = [r.sma for r in indicators.get_bollinger_bands(quotes, 20, 2)]
df['BB_lower_band'] = [r.lower_band for r in indicators.get_bollinger_bands(quotes, 20, 2)]



In [5]:
# Calculate Stochastic Oscillator
stoch_results = indicators.get_stoch(quotes, 14, 3, 3)  # 14 period, 3 period %K smoothing, 3 period %D smoothing
df['Stoch_K'] = [r.k for r in stoch_results]
df['Stoch_D'] = [r.d for r in stoch_results]
df['Buy_Signal'] = ( (df['Close'] <= df['BB_lower_band']) & (df['Stoch_K'] < 20) & (df['Stoch_K'] > df['Stoch_K'].shift(1)) )
df['Sell_Signal'] = ( (df['Close'] >= df['BB_upper_band']) & (df['Stoch_K'] > 80) & (df['Stoch_K'] < df['Stoch_K'].shift(1)) )


# Calculate strategy returns
df['Position'] = 0
df.loc[df['Buy_Signal'], 'Position'] = 1
df.loc[df['Sell_Signal'], 'Position'] = -1
df['Position'] = df['Position'].fillna(method='ffill')
df['Returns'] = df['Close'].pct_change() * df['Position'].shift(1)
df['Strategy_Returns'] = (1 + df['Returns']).cumprod()

C:\Users\jwang\AppData\Local\Temp\ipykernel_6212\886396804.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['Position'] = df['Position'].fillna(method='ffill')


In [6]:
df.tail()   

,Date,Close,High,Low,Open,Volume,BB_upper_band,BB_middle_band,BB_lower_band,Stoch_K,Stoch_D,Buy_Signal,Sell_Signal,Position,Returns,Strategy_Returns
3969,2025-10-14,662.229980,665.830017,653.169983,657.169983,88779600,674.906986,664.626825,654.346664,31.217292,46.074879,False,False,0,-0.0,1.166612
3970,2025-10-15,665.169983,670.229980,658.929993,666.820007,81702600,674.741741,665.017447,655.293153,50.402468,42.199491,False,False,0,0.0,1.166612
3971,2025-10-16,660.640015,668.710022,657.109985,666.820007,110563300,674.732794,665.027997,655.323200,46.612850,42.744203,False,False,0,-0.0,1.166612
3972,2025-10-17,664.390015,665.760010,658.140015,659.500000,96386200,674.753061,665.062497,655.371933,50.023612,49.012977,False,False,0,0.0,1.166612
3973,2025-10-20,671.299988,672.210022,667.270020,667.320007,54272346,675.328276,665.285495,655.242714,59.703083,52.113182,False,False,0,0.0,1.166612


In [7]:
# Bollinger Bands
if __name__ == '__main__':

    chart = Chart(title="Bollinger Bands", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for EMAs
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_line=False, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_line=False, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_line=False, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])

chart.show(block = True)

In [8]:
# Create and display the chart with both indicators
if __name__ == '__main__':
    # Main price chart with Bollinger Bands
    chart = Chart(title="Bollinger Bands + Stochastic Strategy", maximize=True, inner_height=0.8)
    chart.legend(visible=True, color_based_on_candle=True)
    
    # Set the main candlestick data
    chart.set(df)
    
    # Add Bollinger Bands
    upper_band_line = chart.create_line('BB_upper_band', color="#ff0000", width=1, price_line=False, price_label=False)
    upper_band_line.set(df[['Date', 'BB_upper_band']])
    middle_band_line = chart.create_line('BB_middle_band', color="#f3bd0b", width=1, price_line=False, price_label=False)
    middle_band_line.set(df[['Date', 'BB_middle_band']])
    lower_band_line = chart.create_line('BB_lower_band', color="#00ff00", width=1, price_line=False, price_label=False)
    lower_band_line.set(df[['Date', 'BB_lower_band']])
       
          
    # Add Stochastic Oscillator as subchart
    stoch_chart = chart.create_subchart(width=1.0, height=0.2, sync=True)
    stoch_k = stoch_chart.create_line('Stoch_K', color='#2962FF')
    stoch_k.set(df[['Date', 'Stoch_K']])
    stoch_d = stoch_chart.create_line('Stoch_D', color='#FF6D00')
    stoch_d.set(df[['Date', 'Stoch_D']])
    
    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        buy_signal = df.iloc[i]['Buy_Signal']
        sell_signal = df.iloc[i]['Sell_Signal']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (EMA 12 crosses above EMA 25)
        if buy_signal == 1:
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (EMA 12 crosses below EMA 25)
        elif sell_signal == -1:
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)
    
    
    chart.show(block=True)

In [9]:
# Analyze strategy performance
print("Strategy Performance Analysis:")
print("-" * 30)

# Calculate key metrics
total_trades = len(df[df['Position'] != df['Position'].shift(1)]) - 1
winning_trades = len(df[df['Returns'] > 0])
losing_trades = len(df[df['Returns'] < 0])
win_rate = winning_trades / (winning_trades + losing_trades) * 100

# Calculate returns
total_return = (df['Strategy_Returns'].iloc[-1] - 1) * 100
buy_hold_return = ((df['Close'].iloc[-1] / df['Close'].iloc[0]) - 1) * 100

print(f"Total Number of Trades: {total_trades}")
print(f"Winning Trades: {winning_trades}")
print(f"Losing Trades: {losing_trades}")
print(f"Win Rate: {win_rate:.2f}%")
print(f"Strategy Total Return: {total_return:.2f}%")
print(f"Buy & Hold Return: {buy_hold_return:.2f}%")

Strategy Performance Analysis:
------------------------------
Total Number of Trades: 104
Winning Trades: 35
Losing Trades: 24
Win Rate: 59.32%
Strategy Total Return: 16.66%
Buy & Hold Return: 687.18%
